# files

Some file handling, searching, recognizing tools.


## visualizing files



### create a word cloud of imports



#### the code

project word-cloud?



In [ ]:
var importer = require('../Core');
var path = require('path');
    
function projectWordCloud(project) {
    var d3CloudToSVG = importer.import('d3.ipynb[create word-cloud]');
    var relativePaths = importer.import('relative paths and includes', {project});
    
    var words = [];

    function wordCount(r) {
        var words = r['packages'].map(p => p.import)
            .concat(r['packages'].map(p => path.basename(p.file)))
            .concat(r['relatives'].map(r => path.basename(r.import)))
            .concat(r['relatives'].map(r => path.basename(r.file)));
        var wordCount = {};
        words.forEach(w => {
            if (typeof wordCount[w] === 'undefined') {
                wordCount[w] = 15;
            } else {
                wordCount[w]++;
            }
        });
        return Object.keys(wordCount).map((d) => ({text: d, size: wordCount[d]}));
    };

    return relativePaths(project)
        .then(words => d3CloudToSVG(wordCount(words)));
}

module.exports = projectWordCloud;

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
    var project = PROFILE_PATH + '/Documents/portal';

    $$.async()
    projectWordCloud(project)
        .then(svg => $$.svg(svg))
        .catch(e => $$.sendError(e));
}


### create a tree out of imports


#### the code

project imports d3 tree?



In [ ]:
var importer = require('../Core');
var path = require('path');

function icons(c) {
    return c.replace('.component', '\u2699')
           .replace('.module', '\u26E9')
           .replace('.service', '\u26F4')
           .replace('.routing', '\u2697');
}

function wordCount(r) {
    var words = r['packages'].map(p => p.import)
        .concat(r['packages'].map(p => path.basename(p.file)))
        .concat(r['relatives'].map(r => path.basename(r.import)))
        .concat(r['relatives'].map(r => path.basename(r.file)));
    var wordCounts = {};
    words.forEach(w => {
        if (typeof wordCounts[w] === 'undefined') {
            wordCounts[w] = 1;
        } else {
            wordCounts[w]++;
        }
    });
    var resultWords = Object.keys(wordCounts).map((d, i) => ({
        name: icons(d),
        branch: ((/\.component|\.module|\.service|\.routing/ig).exec(d) || {})[0],
        size: wordCounts[d]}));
    var edges = [];
    r['packages'].forEach(p => edges[edges.length] = {
        source: icons(p.import),
        target: icons(path.basename(p.file))
    });
    r['relatives'].forEach(p => edges[edges.length] = {
        source: icons(path.basename(p.file)),
        target: icons(path.basename(p.import))
    });
    return {nodes: resultWords, edges: edges};
};

var formatNodes;
function projectTree(project) {
    var formatNodes = importer.import('d3.ipynb[format tree]');
    var d3TieredPieSVG = importer.import('d3.ipynb[display d3 tree]');
    var projectRelatives = importer.import('relative paths and includes', {project});
    var words = [];
    
    return projectRelatives(project)
        .then(words => {
            var tree = wordCount(words);
            var nodeNames = tree.nodes.map(n => n.name);
            tree.nodes = tree.nodes
                .filter((n, i, arr) => nodeNames.indexOf(n.name) === i)
                .map((n, i) => {
                    Object.assign(n, {index: tree.nodes
                                      .filter(f => f.branch === n.branch).indexOf(n)})
                    return n;
                })
            var edgeNames = tree.edges.map(n => n.source + '/' + n.target);
            tree.edges = tree.edges
                .filter((n, i, arr) => edgeNames.indexOf(n.source + '/' + n.target) === i)
            var root = formatNodes(tree.nodes, tree.edges);
            return d3TieredPieSVG(root);
        })
        .catch(e => console.log(e))
};
module.exports = projectTree;

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
    var project = PROFILE_PATH + '/jupytangular2';

    $$.async()
    projectTree(project)
        .then(svg => $$.svg(svg))
        .catch(e => $$.sendError(e));
}


## listing files

Finding files is easy with minimatch.



### glob files?

Combine minimatch with recursive directory searching.


#### the code


In [1]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var {minimatch} = importer.import('minimatch')
var {ignores} = importer.import('common ignore paths');

// searches project directory recursively for file paths matching the pattern
function globMatch(pattern, cwd, origin) {
    var result = []
    cwd = path.resolve(cwd)
    var files = fs.readdirSync(cwd)
    files.forEach(file => {
        file = '/' + path.relative(origin, path.join(cwd, file))
        
        // skip ignores, don't even search directories
        if(ignores.filter(i => minimatch(file, i)).length > 0) {
            return
        }
        
        var isDirectory = fs.lstatSync(path.join(origin, '.' + file)).isDirectory()
        if(minimatch(file, pattern)) {
            if(isDirectory) {
                // if we are collecting only directories
                if(pattern.substr(-1) === '/') {
                    result.push('.' + file)
                }
            } else {
                // if not collecting directories
                if(pattern.substr(-1) !== '/') {
                    result.push('.' + file)
                }
            }
        }
        
        // recursive matching
        if(isDirectory) {
            result.push.apply(result, globMatch(pattern, '.' + file, origin))
        }
    })
    return result
}

function glob(pattern, cwd) {
    return globMatch(pattern, cwd, cwd)
}

module.exports = {
    glob
}


importing /Users/briancullinan/jupyter_ops/Core/cache.ipynb[1] - 1 cells - cache.ipynb[1]
importing cache all - 1 cell - cache.ipynb[3]
importing list project files - 1 cell - files.ipynb[4]
importing common ignore paths - 1 cell - files.ipynb[5]
importing cache notebook,cell cache,refresh regular cache,update regular cache,update code cell - 5 cells - cache.ipynb[1],cache.ipynb[0],cache.ipynb[13],cache.ipynb[11],cache.ipynb[10]
importing cell cache - 1 cell - cache.ipynb[0]
importing minimatch - 1 cell - files.ipynb[15]
importing expand - 1 cell - files.ipynb[13]
importing mini parser - 1 cell - files.ipynb[14]
importing common ignore paths - 1 cell - files.ipynb[5]


{ glob: [Function: glob] }

#### test glob files



In [6]:
var assert = require('assert')
var importer = require('../Core')
var {glob} = importer.import('glob files')

// test that glob works on the current directory
function testGlobFiles() {
    var results = glob('**index**', '.')
    assert(results.length === 1,
           'index is missing in glob resultss')
    return results
}

module.exports = testGlobFiles

if(typeof $$ !== 'undefined') {
    console.log(testGlobFiles())
}


importing glob files - 1 cell - files.ipynb[2]
importing minimatch - 1 cell - files.ipynb[15]
importing expand - 1 cell - files.ipynb[13]
importing mini parser - 1 cell - files.ipynb[14]
importing common ignore paths - 1 cell - files.ipynb[5]
[ '/Users/briancullinan/jupyter_ops/Core/index.js' ]


### list project files?

List files in a project excluding some excessive searches locations?

TODO: sortLev on all files on no result and find with similar names < 20% difference


#### the code



In [ ]:
var path = require('path');
var {ignores} = importer.import('common ignore paths');
var {glob} = importer.import('glob files')

// TODO: pull all from gitignore library and add to ignore list
//  https://github.com/github/gitignore
// TODO: submit script to github project for fame and fortune!

// But we also want to automatically load projects?
function listInProject(root, match = '{,*,*/,*/*/*,*/*/*/*,*/*/*/*/*}package.json') {
    return glob(match, path.resolve(root))
}

module.exports = {
    listInProject
}


### ignoring files

common ignore paths?

TODO: use the one from github, might need to be reformatted for minimatch

https://github.com/github/gitignore


#### the code

In [ ]:
var ignores = [
    '**/bin/**',
    '**/obj/**',
    '**/dist/**',
    
    '**/*.spec*',
    '**/*.Tests/**',
    '**/typings/**',
    '**/packages/**',
    '**/node_modules/**',
    '**/vendor/**',
//            '**/*test*/*',
//            '**/*Test*/*',
    '**/Downloads/**',
    '**/Library/**',
    '**/Applications/**',
    '**/AppData/**',
    '**/Trash/**',
    '**/proc/**',
    '**/Music/**',
    '**/wp-content/**',
    '**/wp-includes/**',
    '**/Pods/**',
    '**/svn/**',
    '**/lib/**',
    '**/.git/**',
    '**/.vscode/**',
    '**/.npm/**',
    '**/\\.*',
    '**/.*',
    '**/Cache/**',
    '**/Creative Cloud Files/**',
]

module.exports = {
    ignores
}


### listing projects

Detect projects across multiple frameworks.  List them by name for easy lookup.

TODO: get names out of project files, instead of just parent directory.


#### the code

List all projects by names?



In [ ]:
var importer = require('../Core');
var path = require('path');
var listInProject = importer.import('list project files');

// But we also want to automatically load projects?
function listProjects(root, match = '{,*,*/,*/*/*,*/*/*/*,*/*/*/*/*}+(package.json|NuGet.config|*.sln|*.csproj)') {
    var result = {};
    var matches = listInProject(root, match);
    matches.forEach(m => {
        var projectPath = path.resolve(path.dirname(m));
        result[path.basename(projectPath)] = projectPath;
    });
    return result;
};
module.exports = listProjects;



#### test project files list


In [ ]:
var importer = require('../Core');
var {
    listInProject,
    listProjects
} = importer.import(['list project files', 'list all projects by name']);

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;

if(typeof describe !== 'undefined') {

    describe('file system service', () => {
        beforeEach(() => {
        })

        it('should list files quickly', () => {
            var files = listInProject(PROFILE_PATH);
            console.log(files.length);
            assert(files.length > 0);
        })

        it('should ignore hidden directories for better performance', () => {
            var files = listProjects(PROFILE_PATH);
            console.log(files.portal);
            assert(files.jupytangular && files.jupytangular.indexOf('jupytangular') > -1
                    || files.jupytangular2 && files.jupytangular2.indexOf('jupytangular2') > -1, 'could not find own notebook project under the name jupytangular');
        })
    });

}


## fixing files



### find similar filename

Useful when a require or import path changes, this can automatically find files that moved but kept the same name.




#### the code

Find similar filename in project?


In [ ]:
var importer = require('../Core')
var {glob} = importer.import('glob files')

function findSimilarFile(base, project) {
    return new Promise((resolve, reject) => {
        var searchStrings = '**/' + base + '*';
        console.log('Search for ' + searchStrings);
        glob(searchStrings, {
            ignore: ['**/node_modules/**'],
            cwd: project
        }, function (err, matches) {
            var isEmpty = matches.length == 0;
            if (err) {
                return reject(err);
            } else if (isEmpty) {
                console.log('No matches for ' + searchStrings)
                return resolve();
            }

            // TODO: try to build and make sure it works before patching
            resolve(matches[0].replace('.ts', ''));
        });
    });
};

module.exports = findSimilarFile;



### fix all imports in a project


#### the code

fix project paths?

TODO: make this syntax aware using transpiler

TODO: find file based on matching syntax structure

TODO: convert this to a fix for eslint or transpiler


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var {relativeImports} = importer.import('builtin and local modules')

function fixImports(project) {
    var packagePath = path.join(project, 'package.json')
    var codeFiles = listInProject(project, '**/*.+(ts|js|ipynb)')
    codeFiles.forEach(f => {
        var imports = relativeImports(f)
        
        // TODO: find file based on matching syntax structure
        imports.missing.forEach(i => {
            var match = codeFiles
                .filter(f => path.basename(f) === path.basename(i))[0]
            if(match) {
                var newRelative = './' + path.relative(
                    path.dirname(f), path.join(project, match))
                var fixFile = fs.readFileSync(f).toString()
                fs.writeFileSync(f, fixFile.replace(i, newRelative))
            }
        })
        
        // fix package json
        if(fs.existsSync(packagePath)) {
            var packageJson = JSON.parse(fs.readFileSync(packagePath))
            var toAdd = imports.packages
                .filter(p => typeof packageJson.dependencies[p] === 'undefined'
                        && typeof packageJson.devDependencies[p] === 'undefined')
            toAdd.forEach(a => {
                console.log(`adding ${a}`)
                packageJson.dependencies[a] = 'latest'
            })
            if(toAdd.length > 0) {
                fs.writeFileSync(packagePath,
                                 JSON.stringify(packageJson, null, 4))
            }
        }
    })
}

module.exports = {
    fixImports
}


#### TODO: test fix imports

Using memory-fs make a code file, move a reference, check the file was corrected.


### find functions that are too long

(?=\{((?:[^{}]++|\{(?1)\})++)\})

TODO: turn this in to a bs-lint test

TODO: use a recusive decent parser to apply to all languages, test finding long python functions which are white-space delimited


#### the code

Find too long of functions using regexp?



In [ ]:
var importer = require('../Core');
var {listInProject} = importer.import('list project files');

function matchCurlyBraces(contents) {
    var XRegExp = require('xregexp');
    var newMatches = XRegExp.matchRecursive(contents, '{', '}', 'gi', {
        valueNames: ['literal', null, 'value', null]
    }).filter(m => m.name === 'value');
    //console.log(newMatches);
    return newMatches.reduce(
        (arr, m) => arr.concat(matchCurlyBraces(m.value)),
        newMatches);
};

function findLongFunctions(project) {
    var files = listInProject(project, '**/*.+(ts|js|cs)')
    var results = files.map(file => {
        const contents = file.contents.toString();
        try {
            results = results.concat(matchCurlyBraces(contents)
                .map(m => Object.assign(m, {path: file.path})))
        } catch (e) {
            results = results.concat([{
                name: 'value',
                value: file.contents.toString(),
                start: 0,
                end: file.contents.length,
                error: e,
                path: file.path
            }])
        }
    });
    var code = results.map(r => r.value);
    var topLevel = results
        .filter(r => code
            .filter(c => c !== r.value && c.indexOf(r.value) > 0).length === 0)
    results = results
        .filter(m => topLevel.filter(c => c.value === m.value).length === 0)
    code = results.map(r => r.value);
    topLevel = topLevel.concat(results.filter(r => code
        .filter(c => c !== r.value && c.indexOf(r.value) > 0).length === 0))
    results = results
        .filter(m => topLevel.filter(c => c.value === m.value).length === 0)
    return results
        .filter(m => m.value.split(/\n/ig).length > 100)
}

module.exports = findLongFunctions


#### test find too long of functions


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/asm';

if(typeof $$ !== 'undefined') {
    $$.async();
    findLongFunctions(project)
        .then(e => $$.sendResult(e.map(m => m.path)))
        .catch(e => $$.sendError(e))

}


## file tools

### minimatch

Just the path matching, none of the options or bells and whistles.

Derived from: https://github.com/isaacs/minimatch/blob/master/minimatch.js


#### expand?


In [ ]:
var concatMap = require('concat-map');
var balanced = require('balanced-match');

module.exports = expandTop;

var escSlash = '\0SLASH'+Math.random()+'\0';
var escOpen = '\0OPEN'+Math.random()+'\0';
var escClose = '\0CLOSE'+Math.random()+'\0';
var escComma = '\0COMMA'+Math.random()+'\0';
var escPeriod = '\0PERIOD'+Math.random()+'\0';

function numeric(str) {
  return parseInt(str, 10) == str
    ? parseInt(str, 10)
    : str.charCodeAt(0);
}

function escapeBraces(str) {
  return str.split('\\\\').join(escSlash)
            .split('\\{').join(escOpen)
            .split('\\}').join(escClose)
            .split('\\,').join(escComma)
            .split('\\.').join(escPeriod);
}

function unescapeBraces(str) {
  return str.split(escSlash).join('\\')
            .split(escOpen).join('{')
            .split(escClose).join('}')
            .split(escComma).join(',')
            .split(escPeriod).join('.');
}


// Basically just str.split(","), but handling cases
// where we have nested braced sections, which should be
// treated as individual members, like {a,{b,c},d}
function parseCommaParts(str) {
  if (!str)
    return [''];

  var parts = [];
  var m = balanced('{', '}', str);

  if (!m)
    return str.split(',');

  var pre = m.pre;
  var body = m.body;
  var post = m.post;
  var p = pre.split(',');

  p[p.length-1] += '{' + body + '}';
  var postParts = parseCommaParts(post);
  if (post.length) {
    p[p.length-1] += postParts.shift();
    p.push.apply(p, postParts);
  }

  parts.push.apply(parts, p);

  return parts;
}

function expandTop(str) {
  if (!str)
    return [];

  // I don't know why Bash 4.3 does this, but it does.
  // Anything starting with {} will have the first two bytes preserved
  // but *only* at the top level, so {},a}b will not expand to anything,
  // but a{},b}c will be expanded to [a}c,abc].
  // One could argue that this is a bug in Bash, but since the goal of
  // this module is to match Bash's rules, we escape a leading {}
  if (str.substr(0, 2) === '{}') {
    str = '\\{\\}' + str.substr(2);
  }

  return expand(escapeBraces(str), true).map(unescapeBraces);
}

function identity(e) {
  return e;
}

function embrace(str) {
  return '{' + str + '}';
}
function isPadded(el) {
  return /^-?0\d/.test(el);
}

function lte(i, y) {
  return i <= y;
}
function gte(i, y) {
  return i >= y;
}

function expand(str, isTop) {
  var expansions = [];

  var m = balanced('{', '}', str);
  if (!m || /\$$/.test(m.pre)) return [str];

  var isNumericSequence = /^-?\d+\.\.-?\d+(?:\.\.-?\d+)?$/.test(m.body);
  var isAlphaSequence = /^[a-zA-Z]\.\.[a-zA-Z](?:\.\.-?\d+)?$/.test(m.body);
  var isSequence = isNumericSequence || isAlphaSequence;
  var isOptions = m.body.indexOf(',') >= 0;
  if (!isSequence && !isOptions) {
    // {a},b}
    if (m.post.match(/,.*\}/)) {
      str = m.pre + '{' + m.body + escClose + m.post;
      return expand(str);
    }
    return [str];
  }

  var n;
  if (isSequence) {
    n = m.body.split(/\.\./);
  } else {
    n = parseCommaParts(m.body);
    if (n.length === 1) {
      // x{{a,b}}y ==> x{a}y x{b}y
      n = expand(n[0], false).map(embrace);
      if (n.length === 1) {
        var post = m.post.length
          ? expand(m.post, false)
          : [''];
        return post.map(function(p) {
          return m.pre + n[0] + p;
        });
      }
    }
  }

  // at this point, n is the parts, and we know it's not a comma set
  // with a single entry.

  // no need to expand pre, since it is guaranteed to be free of brace-sets
  var pre = m.pre;
  var post = m.post.length
    ? expand(m.post, false)
    : [''];

  var N;

  if (isSequence) {
    var x = numeric(n[0]);
    var y = numeric(n[1]);
    var width = Math.max(n[0].length, n[1].length)
    var incr = n.length == 3
      ? Math.abs(numeric(n[2]))
      : 1;
    var test = lte;
    var reverse = y < x;
    if (reverse) {
      incr *= -1;
      test = gte;
    }
    var pad = n.some(isPadded);

    N = [];

    for (var i = x; test(i, y); i += incr) {
      var c;
      if (isAlphaSequence) {
        c = String.fromCharCode(i);
        if (c === '\\')
          c = '';
      } else {
        c = String(i);
        if (pad) {
          var need = width - c.length;
          if (need > 0) {
            var z = new Array(need + 1).join('0');
            if (i < 0)
              c = '-' + z + c.slice(1);
            else
              c = z + c;
          }
        }
      }
      N.push(c);
    }
  } else {
    N = concatMap(n, function(el) { return expand(el, false) });
  }

  for (var j = 0; j < N.length; j++) {
    for (var k = 0; k < post.length; k++) {
      var expansion = pre + N[j] + post[k];
      if (!isTop || isSequence || expansion)
        expansions.push(expansion);
    }
  }

  return expansions;
}


#### mini parser?


In [ ]:
var reSpecials = charSet('().*{}+?[]^$\\!')

// replace stuff like \* with *
function globUnescape (s) {
  return s.replace(/\\(.)/g, '$1')
}

function charSet (s) {
  return s.split('').reduce(function (set, c) {
    set[c] = true
    return set
  }, {})
}

var SUBPARSE = {}
function parse (pattern, isSub) {
  if (pattern.length > 1024 * 64) {
    throw new TypeError('pattern is too long')
  }

  // shortcuts
  if (pattern === '**') return {glob: true}
  if (pattern === '') return ''

  var re = ''
  var hasMagic = false
  var escaping = false
  // ? => one single character
  var patternListStack = []
  var negativeLists = []
  var stateChar
  var inClass = false
  var reClassStart = -1
  var classStart = -1
  // . and .. never match anything that doesn't start with .,
  // even when options.dot is set.
  var patternStart = pattern.charAt(0) === '.' ? '' // anything
  // not (start or / followed by . or .. followed by / or end)
  : '(?!(?:^|\\\/)\\.{1,2}(?:$|\\\/))'

  function clearStateChar () {
    if (stateChar) {
      // we had some state-tracking character
      // that wasn't consumed by this pass.
      switch (stateChar) {
        case '*':
          re += '[^/]*?'
          hasMagic = true
        break
        case '?':
          re += '[^/]'
          hasMagic = true
        break
        default:
          re += '\\' + stateChar
        break
      }
      stateChar = false
    }
  }

  for (var i = 0, len = pattern.length, c
    ; (i < len) && (c = pattern.charAt(i))
    ; i++) {
    // skip over any that are escaped.
    if (escaping && reSpecials[c]) {
      re += '\\' + c
      escaping = false
      continue
    }

    switch (c) {
      case '/':
        // completely not allowed, even escaped.
        // Should already be path-split by now.
        return false

      case '\\':
        clearStateChar()
        escaping = true
      continue

      // the various stateChar values
      // for the "extglob" stuff.
      case '?':
      case '*':
      case '+':
      case '@':
      case '!':
        // all of those are literals inside a class, except that
        // the glob [!a] means [^a] in regexp
        if (inClass) {
          if (c === '!' && i === classStart + 1) c = '^'
          re += c
          continue
        }

        // if we already have a stateChar, then it means
        // that there was something like ** or +? in there.
        // Handle the stateChar, then proceed with this one.
        clearStateChar()
        stateChar = c
        // if extglob is disabled, then +(asdf|foo) isn't a thing.
        // just clear the statechar *now*, rather than even diving into
        // the patternList stuff.
      continue

      case '(':
        if (inClass) {
          re += '('
          continue
        }

        if (!stateChar) {
          re += '\\('
          continue
        }

        patternListStack.push({
          type: stateChar,
          start: i - 1,
          reStart: re.length,
          open: plTypes[stateChar].open,
          close: plTypes[stateChar].close
        })
        // negation is (?:(?!js)[^/]*)
        re += stateChar === '!' ? '(?:(?!(?:' : '(?:'
        stateChar = false
      continue

      case ')':
        if (inClass || !patternListStack.length) {
          re += '\\)'
          continue
        }

        clearStateChar()
        hasMagic = true
        var pl = patternListStack.pop()
        // negation is (?:(?!js)[^/]*)
        // The others are (?:<pattern>)<type>
        re += pl.close
        if (pl.type === '!') {
          negativeLists.push(pl)
        }
        pl.reEnd = re.length
      continue

      case '|':
        if (inClass || !patternListStack.length || escaping) {
          re += '\\|'
          escaping = false
          continue
        }

        clearStateChar()
        re += '|'
      continue

      // these are mostly the same in regexp and glob
      case '[':
        // swallow any state-tracking char before the [
        clearStateChar()

        if (inClass) {
          re += '\\' + c
          continue
        }

        inClass = true
        classStart = i
        reClassStart = re.length
        re += c
      continue

      case ']':
        //  a right bracket shall lose its special
        //  meaning and represent itself in
        //  a bracket expression if it occurs
        //  first in the list.  -- POSIX.2 2.8.3.2
        if (i === classStart + 1 || !inClass) {
          re += '\\' + c
          escaping = false
          continue
        }

        // handle the case where we left a class open.
        // "[z-a]" is valid, equivalent to "\[z-a\]"
        if (inClass) {
          // split where the last [ was, make sure we don't have
          // an invalid re. if so, re-walk the contents of the
          // would-be class to re-translate any characters that
          // were passed through as-is
          // TODO: It would probably be faster to determine this
          // without a try/catch and a new RegExp, but it's tricky
          // to do safely.  For now, this is safe and works.
          var cs = pattern.substring(classStart + 1, i)
          try {
            RegExp('[' + cs + ']')
          } catch (er) {
            // not a valid class!
            var sp = parse(cs, SUBPARSE)
            re = re.substr(0, reClassStart) + '\\[' + sp[0] + '\\]'
            hasMagic = hasMagic || sp[1]
            inClass = false
            continue
          }
        }

        // finish up the class.
        hasMagic = true
        inClass = false
        re += c
      continue

      default:
        // swallow any state char that wasn't consumed
        clearStateChar()

        if (escaping) {
          // no need
          escaping = false
        } else if (reSpecials[c]
          && !(c === '^' && inClass)) {
          re += '\\'
        }

        re += c

    } // switch
  } // for

  // handle the case where we left a class open.
  // "[abc" is valid, equivalent to "\[abc"
  if (inClass) {
    // split where the last [ was, and escape it
    // this is a huge pita.  We now have to re-walk
    // the contents of the would-be class to re-translate
    // any characters that were passed through as-is
    cs = pattern.substr(classStart + 1)
    sp = parse(cs, SUBPARSE)
    re = re.substr(0, reClassStart) + '\\[' + sp[0]
    hasMagic = hasMagic || sp[1]
  }

  // handle the case where we had a +( thing at the *end*
  // of the pattern.
  // each pattern list stack adds 3 chars, and we need to go through
  // and escape any | chars that were passed through as-is for the regexp.
  // Go through and escape them, taking care not to double-escape any
  // | chars that were already escaped.
  for (pl = patternListStack.pop(); pl; pl = patternListStack.pop()) {
    var tail = re.slice(pl.reStart + pl.open.length)
    // maybe some even number of \, then maybe 1 \, followed by a |
    tail = tail.replace(/((?:\\{2}){0,64})(\\?)\|/g, function (_, $1, $2) {
      if (!$2) {
        // the | isn't already escaped, so escape it.
        $2 = '\\'
      }

      // need to escape all those slashes *again*, without escaping the
      // one that we need for escaping the | character.  As it works out,
      // escaping an even number of slashes can be done by simply repeating
      // it exactly after itself.  That's why this trick works.
      //
      // I am sorry that you have to see this.
      return $1 + $1 + $2 + '|'
    })

    var t = pl.type === '*' ? star
      : pl.type === '?' ? qmark
      : '\\' + pl.type

    hasMagic = true
    re = re.slice(0, pl.reStart) + t + '\\(' + tail
  }

  // handle trailing things that only matter at the very end.
  clearStateChar()
  if (escaping) {
    // trailing \\
    re += '\\\\'
  }

  // only need to apply the nodot start if the re starts with
  // something that could conceivably capture a dot
  var addPatternStart = false
  switch (re.charAt(0)) {
    case '.':
    case '[':
    case '(': addPatternStart = true
  }

  // Hack to work around lack of negative lookbehind in JS
  // A pattern like: *.!(x).!(y|z) needs to ensure that a name
  // like 'a.xyz.yz' doesn't match.  So, the first negative
  // lookahead, has to look ALL the way ahead, to the end of
  // the pattern.
  for (var n = negativeLists.length - 1; n > -1; n--) {
    var nl = negativeLists[n]

    var nlBefore = re.slice(0, nl.reStart)
    var nlFirst = re.slice(nl.reStart, nl.reEnd - 8)
    var nlLast = re.slice(nl.reEnd - 8, nl.reEnd)
    var nlAfter = re.slice(nl.reEnd)

    nlLast += nlAfter

    // Handle nested stuff like *(*.js|!(*.json)), where open parens
    // mean that we should *not* include the ) in the bit that is considered
    // "after" the negated section.
    var openParensBefore = nlBefore.split('(').length - 1
    var cleanAfter = nlAfter
    for (i = 0; i < openParensBefore; i++) {
      cleanAfter = cleanAfter.replace(/\)[+*?]?/, '')
    }
    nlAfter = cleanAfter

    var dollar = ''
    if (nlAfter === '' && isSub !== SUBPARSE) {
      dollar = '$'
    }
    var newRe = nlBefore + nlFirst + nlAfter + dollar + nlLast
    re = newRe
  }

  // if the re is not "" at this point, then we need to make sure
  // it doesn't match against an empty path part.
  // Otherwise a/* will match a/, which it should not.
  if (re !== '' && hasMagic) {
    re = '(?=.)' + re
  }

  if (addPatternStart) {
    re = patternStart + re
  }

  // parsing just a piece of a larger pattern.
  if (isSub === SUBPARSE) {
    return [re, hasMagic]
  }

  // skip the regexp for non-magical patterns
  // unescape anything in it, though, so that it'll be
  // an exact match against a file etc.
  if (!hasMagic) {
    return globUnescape(pattern)
  }

  try {
    var regExp = new RegExp('^' + re + '$', 'i')
  } catch (er) {
    // If it was an invalid regular expression, then it can't match
    // anything.  This trick looks for a character after the end of
    // the string, which is of course impossible, except in multi-line
    // mode, but it's not a /m regex.
    return new RegExp('$.')
  }

  regExp._glob = pattern
  regExp._src = re

  return regExp
}

module.exports = parse


#### minimatch?



In [ ]:
var importer = require('../Core')
var expand = importer.import('expand')
var parse = importer.import('mini parser')

function regExpEscape (s) {
  return s.replace(/[-[\]{}()*+?.,\\^$|#\s]/g, '\\$&')
}

// convert minimatch expression to regular expression
function minimatch (filename, pattern) {
    // expand braces
    var set = expand(pattern)

        // matching patterns.
        .map(function (s) {
            return s.split(/\/+/)
        })

        // glob --> regexps
        .map(function (s, si, set) {
            return s.map(parse)
        })

        // filter out everything that didn't compile properly.
        .filter(function (s) {
            return s.indexOf(false) === -1
        })

        // convert the sets to regular expressions
        .map(function (pattern) {
            return pattern.map(function (p) {
                return (typeof p.glob === 'boolean') 
                    ? '(?:(?!(?:\\\/|^)(?:\\.{1,2})($|\\\/)).)*?'
                    : (typeof p === 'string') ? regExpEscape(p)
                    : p._src
            }).join('\\\/')
        }).join('|')
    try {
        var re = new RegExp('^(?:' + set + ')$', 'i')
        return filename.match(re) !== null
    } catch (ex) {
        throw ex
    }
}

module.exports = {
    minimatch
}


#### test minimatch



In [28]:
var importer = require('../Core')
var {minimatch} = importer.import('minimatch')
//var mini = require('minimatch')

console.log(minimatch('/some/file/to/test', '**/file/**'))
console.log(minimatch('/node_modules/@firebase', '**/node_modules/**'))
//mini.makeRe('**/node_modules/**', {dot: true, regexp: true})
//console.log(mini('/node_modules/@firebase', '**/node_modules/**'))

/*
expected output
true
true
*/


importing minimatch - 1 cell - files.ipynb[15]
true
true
true


### match filename?

Minimatching from an object full of file names and their output destination.


In [ ]:
var importer = require('../Core')
var {minimatch} = importer.import('minimatch')

var getDirectory = (match) => match[match.length-1] === '/'
    ? match
    : (fs.existsSync(match) && fs.lstatSync(match).isDirectory()
       ? match
       : path.dirname(match));


function matchFilename(filename, matchOutput, projectOutput) {
    return Object.keys(matchOutput)
    .filter(match => minimatch(filename, match))
    .map(match => {
        var projectMatch = path.join(projectOutput, matchOutput[match])
        var dir = getDirectory(projectMatch);
        return path.join(dir, path.basename(projectMatch) === path.basename(dir)
            ? filename
            : path.basename(matchOutput[match]));
    })
}

module.exports = {
    matchFilename,
    getDirectory
}


### make directories recursively

mkdirp?


In [ ]:
var fs = require('fs');
var path = require('path');

function mkdirpSync(p) {
    if(!fs.existsSync(p)) {
        try {
            fs.mkdirSync(p);
        } catch (e) {
            if(e.code === 'ENOENT') {
                mkdirpSync(path.dirname(p));
                fs.mkdirSync(p);
            }
        }
    }
}

module.exports = mkdirpSync;


### mocking file-system calls

This can be useful if a library doesn't natively support streams or putting content in to memory instead of directly on disk.



#### the code

How to use memory-fs and rewire to audit cli events?



In [ ]:
var path = require('path');
var mock = require('mock-require');
var memfs = require('memfs');
var unionfs = require('unionfs');
var fs = require('fs');
var ts = require('typescript');

// TODO: use https://github.com/jhnns/rewire for replacing variables, something like https://github.com/mariocasciaro/object-path?

function mockTypescriptFs(root, data) {
    var mem = new memfs.Volume;

    mem.mountSync(root, data);

    // Create a union of two file systems:
    unionfs
        .use(fs)
        .use(mem)
        .replace(fs);

    var tsMock = Object.assign({}, ts);
    var toMock = {
        readFile: 'readFileSync',
        realpath: 'realpathSync',
        writeFile: 'writeFileSync',
        fileExists: 'existsSync',
        directoryExists: 'existsSync',
        createDirectory: 'mkdirSync',
        getDirectories: 'readdirSync'
    };
    Object.keys(toMock)
        .forEach(k => {
            if (typeof mem[toMock[k]] === 'function') {
                tsMock.sys[k] = function () {
                    // typescript.sys functions return defined on error
                    try {
                        return mem[toMock[k]].apply(mem, arguments);
                    } catch (e) {
                        return;
                    }
                };
            }
        });

    mock('typescript', tsMock);
    return mock;
};
module.exports = mockTypescriptFs;

// TODO: search github for projects like graceful-fs and then search google for projects that use graceful-fs and see if we can record some filesystem activity



## TODO: use static code analysis to find consecutive words in template files?

https://stackoverflow.com/questions/18717536/in-python-how-can-i-distinguish-between-a-human-readible-word-and-random-string

TODO: combine with DOM minimizing service from recording app?

TODO: use to find the main content of the page for scraping.


In [ ]:
var fs = require('fs');
var importer = require('../Core')
var {glob} = importer.import('glob files')
fs.readdirSync('.');
var PROJECT_PATH = '/Users/briancullinan/Google Drive/Colab Notebooks'
glob.sync('**/*.ipynb', {cwd: PROJECT_PATH}).forEach(f => {
    const nb = JSON.parse(fs.readFileSync(path.join(PROJECT_PATH, f)).toString());
    nb.nbformat_minor = 0;
    /*
    nb.metadata = {
        colab: {
          "name": path.basename(f),
          "version": "0.3.2",
          "views": {},
          "default_view": {},
          "provenance": []
        },
        kernelspec: {
          "name": "javascript",
          "display_name": "Javascript"
        }
    };
    */
    fs.writeFileSync(path.join(PROJECT_PATH, f), JSON.stringify(nb, null, 2));
});


## TODO:

TODO: make a utulity for displaying files that match specific code.

TODO: make a utility for loading file information like PECL ID3 from PHP.
